In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')


In [3]:
df = pd.read_csv("StudentsPerformance.csv")
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [4]:
X = df.drop(columns = ['math score'], axis=1)
y = df['math score']

In [5]:
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [6]:
num_feature   = X.select_dtypes(exclude="object").columns
cat_feature = X.select_dtypes(include="object").columns


from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_feature = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OnehotEncoder", oh_transformer, cat_feature),
        ("StandardScaler", numeric_feature, num_feature)
    ]
    
)

In [7]:
X = preprocessor.fit_transform(X)

In [8]:
num_feature

Index(['reading score', 'writing score'], dtype='object')

In [9]:
X.shape

(1000, 19)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [15]:
def evaluate_model(true, predicted):
    mse = mean_squared_error(true, predicted)
    r2 = r2_score(true, predicted)
    return(mse, r2)



In [18]:
models = {
    'RandomForestRegressor' : RandomForestRegressor(),
    'XGBoostRegressor' : XGBRegressor()
}

model_list  = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    model_train_mse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print("Model performance for training")
    print(" - MSE:  {:.4f}".format(model_train_mse))
    print(" - r2 score : {:.4f}".format(model_train_r2))
    
    print("**************")
    
    print("Model performance for testing")
    print(" - MSE:  {:.4f}".format(model_test_mse))
    print(" - r2 score : {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    print('\n')

RandomForestRegressor
Model performance for training
 - MSE:  5.4373
 - r2 score : 0.9759
**************
Model performance for testing
 - MSE:  36.1754
 - r2 score : 0.8513


XGBoostRegressor
Model performance for training
 - MSE:  1.0146
 - r2 score : 0.9955
**************
Model performance for testing
 - MSE:  41.9037
 - r2 score : 0.8278


